In [1]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv("../Data/train-balanced-sarcasm.csv")

In [13]:
df_new = df[['parent_comment','comment','label']]
df_new

,parent_comment,comment,label
0,"Yeah, I get that argument. At this point, I'd ...",NC and NH.,0
1,The blazers and Mavericks (The wests 5 and 6 s...,You do know west teams play against west teams...,0
2,They're favored to win.,"They were underdogs earlier today, but since G...",0
3,deadass don't kill my buzz,"This meme isn't funny none of the ""new york ni...",0
4,Yep can confirm I saw the tool they use for th...,I could use one of those tools.,0
5,do you find ariana grande sexy ?,"I don't pay attention to her, but as long as s...",0
6,What's your weird or unsettling Trick or Treat...,Trick or treating in general is just weird...,0
7,Probably Sephiroth. I refuse to taint his grea...,Blade Mastery+Masamune or GTFO!,0
8,What to upgrade? I have $500 to spend (mainly ...,"You don't have to, you have a good build, buy ...",0
9,Probably count Kanye out Since the rest of his...,I would love to see him at lolla.,0


In [30]:
sentences = []
for s in df_new['parent_comment'][:1010826]:
    sentences.append(s)
for s in df_new['comment'][:1010826]:
    sentences.append(s)

In [35]:
words = []
print(sentences[:100])

AttributeError: 'list' object has no attribute 'lower'